# Angestrebter Studienabschluss und Fachbereichsgröße

## Import Desing-Template

In [ ]:
import plotly.io as pio

# Template
infoviz_template = dict(
    layout=dict(
        template="plotly_white",
        title=dict(
            font=dict(size=20, family="Arial", weight="bold", color="black"),
            xanchor="left",  
            xref="paper",
            x=0,
            subtitle=dict(
                text="",
                font=dict(color="gray", size=13),
            ),
        ),
        xaxis=dict(
            showgrid=False,
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside bottom"
        ),
        yaxis=dict(
            showgrid=True, gridcolor="lightgrey",
            zerolinecolor="lightgrey",
            tickfont=dict(color="grey", size=12),
            title_font=dict(color="grey", weight="bold", size=13),
            title_standoff=15,
            ticklabelstandoff=10,
            ticklabelposition="outside left"
        ),
    )
)
pio.templates["infoviz"] = infoviz_template

## Daten einlesen

In [211]:
import pandas as pd

# Datei einlesen (ersetze 'data.xlsx' durch den tatsächlichen Dateinamen)
file_path = "data.xlsx"

# Alle Sheets einlesen, aber das erste Blatt ignorieren
sheets = pd.read_excel(file_path, sheet_name=None, engine="openpyxl")

# Erstes Blatt entfernen
sheets.pop(next(iter(sheets)))

# Container für die bereinigten DataFrames
cleaned_sheets = {}

# Durch alle Jahre iterieren und DataFrames bereinigen
for jahr, df in sheets.items():
    df.columns = df.columns.astype(str).str.strip()  # Spaltennamen bereinigen
    cleaned_sheets[jahr] = df  # Speichern der bereinigten Daten

In [212]:
abschluss_anteile = {}

for jahr, df in cleaned_sheets.items():
    # Spalten automatisch nach Bachelor und Master aufteilen
    bachelor_columns = [col for col in df.columns if "Bachelor" in col]
    master_columns = [col for col in df.columns if "Master" in col]

    # Extrahiere die Gesamtanzahl der Studierenden pro Abschluss
    total_bachelor = df.loc[df["Category"] == "Anzahl Studierende", bachelor_columns].sum().sum()
    total_master = df.loc[df["Category"] == "Anzahl Studierende", master_columns].sum().sum()

    total_ruecklauf_bachelor = df.loc[df["Category"] == "Anzahl Teilnahmen an Studierendenbefragung", bachelor_columns].sum().sum()
    total_ruecklauf_master = df.loc[df["Category"] == "Anzahl Teilnahmen an Studierendenbefragung", master_columns].sum().sum()

    # Prozentanteile berechnen
    total_students = total_bachelor + total_master
    anteil_bachelor = (total_bachelor / total_students * 100) if total_students > 0 else 0
    anteil_master = (total_master / total_students * 100) if total_students > 0 else 0

    # Gesamtzahlen berechnen
    gesamt_ruecklauf = total_ruecklauf_bachelor + total_ruecklauf_master
    gesamt_ruecklaufquote = gesamt_ruecklauf / total_students * 100
    nicht_teilgenommen = total_students - gesamt_ruecklauf
    

    # Ergebnisse speichern
    abschluss_anteile[jahr] = {
        "Absolute Anzahl Bachelor": total_bachelor,
        "Absolute Anzahl Master": total_master,
        "Absoluter Rücklauf Bachelor": total_ruecklauf_bachelor,
        "Absoluter Rücklauf Master": total_ruecklauf_master,
        "Gesamtzahl Studierende": total_bachelor + total_master,
        "Gesamter Rücklauf": gesamt_ruecklauf,
        "Gesamte Rücklaufquote": gesamt_ruecklaufquote,
        "Gesamt nicht teilgenommen": nicht_teilgenommen,
        "Anteil Bachelor (%)": anteil_bachelor,
        "Anteil Master (%)": anteil_master,
    }

# DataFrame erstellen
abschluss_df = pd.DataFrame.from_dict(abschluss_anteile, orient="index").reset_index().rename(columns={"index": "Jahr"})
abschluss_df["Jahr"] = abschluss_df["Jahr"].astype(int)
abschluss_df.head()  # Zur Kontrolle anzeigen

,Jahr,Absolute Anzahl Bachelor,Absolute Anzahl Master,Absoluter Rücklauf Bachelor,Absoluter Rücklauf Master,Gesamtzahl Studierende,Gesamter Rücklauf,Gesamte Rücklaufquote,Gesamt nicht teilgenommen,Anteil Bachelor (%),Anteil Master (%)
0,2013,1406.0,801.0,409.0,231.0,2207.0,640.0,28.998641,1567.0,63.706389,36.293611
1,2014,1580.0,767.0,453.0,241.0,2347.0,694.0,29.569663,1653.0,67.319983,32.680017
2,2015,1710.0,747.0,433.0,198.0,2457.0,631.0,25.681726,1826.0,69.597070,30.402930
3,2016,1795.0,967.0,494.0,205.0,2762.0,699.0,25.307748,2063.0,64.989138,35.010862
4,2017,1848.0,1236.0,491.0,267.0,3084.0,758.0,24.578470,2326.0,59.922179,40.077821


## Entwicklung aller Abschlüsse

In [305]:
import plotly.express as px

fig = px.line(
    abschluss_df,
    x="Jahr",
    y="Gesamte Rücklaufquote",
    markers=True,
    title="<span style='color:#73c6e9;'>Rücklaufquote</span> des Fachbereichs 09 nimmt seit Jahren ab",
    subtitle="Entwicklung der Studierendenbefragung zwischen 2013 und 2024",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3)) 

fig.update_layout(
    yaxis=dict(range=[0, 45], title="Rücklaufquote %"),
    xaxis=dict(range=[2012.9, 2024.2], dtick=1, title=""),
)

fig.show()

In [419]:
color1, color2 = px.colors.qualitative.Safe[4], px.colors.qualitative.Safe[6]
fig = px.bar(
    abschluss_df,
    x="Jahr",
    y=["Gesamter Rücklauf", "Gesamt nicht teilgenommen"],
    title=rf"<b>Schrumpfen des Fachbereichs trifft </b><span style='color:{color1};'>Teilnehmende</span> <b>und </b><span style='color:{color2};'>nicht Teilnehmende</span>",
    subtitle="Entwicklung der Studierendenbefragung im FB09 zwischen 2013 und 2024",
    color_discrete_sequence=[color1, color2],
    template="infoviz"
)

fig.update_layout(
    yaxis=dict(range=[0, 3600], title="Anzahl Studierende"),
    xaxis=dict(dtick=1, title=""),
    showlegend=False
)

fig.show()

## Abschlussarten

In [240]:
fig = px.line(
    abschluss_df,
    x="Jahr",
    y=["Absoluter Rücklauf Bachelor", "Absoluter Rücklauf Master"],
    markers=True,
    title="<b>Teilnehmeranzahl von </b><span style='color:#73c6e9;'>Bachelor</span> <b>und </b><span style='color:#cc5b6e;'>Master</span> <b>mittlerweile auf ähnlichem Niveau </b>",
    subtitle="Entwicklung der Studierendenbefragung im FB09 zwischen 2013 und 2024",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3))

fig.update_layout(
    yaxis=dict(range=[0, 650], title="Anzahl Teilnehmende"),
    xaxis=dict(dtick=1, title="", range=[2012.9, 2024.2]),
    showlegend=False
)

fig.show()

In [304]:
import plotly.express as px

fig = px.line(
    abschluss_df,
    x="Jahr",
    y=["Absolute Anzahl Bachelor", "Absolute Anzahl Master"],
    markers=True,
    title="<b>Rückgang des Fachbereichs trifft </b><span style='color:#73c6e9;'>Bachelor</span> <b>härter als </b><span style='color:#cc5b6e;'>Master</span>",
    subtitle="Entwicklung der Studierendenbefragung im FB09 zwischen 2013 und 2024",
    color_discrete_sequence=px.colors.qualitative.Safe,
    template="infoviz"
)

fig.update_traces(line=dict(width=3))  # Linie dicker machen

fig.update_layout(
    yaxis=dict(range=[0, 2100], title="Anzahl Studierende"),
    xaxis=dict(dtick=1, title="", range=[2012.9, 2024.2]),
    showlegend=False
)

fig.show()

## Verteilung der Studiengänge

In [442]:

df_2024 = cleaned_sheets["2024"]

# Studiengänge automatisch erkennen (Bachelor und Master)
bachelor_studiengaenge = [col for col in df_2024.columns if "Bachelor" in col]
master_studiengaenge = [col for col in df_2024.columns if "Master" in col]

# Extrahiere die Anzahl der Studierenden pro Studiengang
studiengang_verteilung = df_2024[df_2024["Category"] == "Anzahl Studierende"][bachelor_studiengaenge + master_studiengaenge].T
studiengang_verteilung.columns = ["Anzahl Studierende"]
studiengang_verteilung["Studiengang"] = studiengang_verteilung.index

# Hinzufügen der übergeordneten Kategorie ("Bachelor" oder "Master")
studiengang_verteilung["Studienabschluss"] = studiengang_verteilung["Studiengang"].apply(
    lambda x: "Bachelor" if "Bachelor" in x else "Master"
)

studiengang_verteilung["Studiengang"]=studiengang_verteilung["Studiengang"].apply(lambda x: x.replace("Bachelor", "B.Sc.").replace("Master", "M.Sc."))
studiengang_verteilung["Studiengang"] = studiengang_verteilung["Studiengang"].replace(
    "M.Sc. Informationstechnologie in den Agrar- und Umweltwissenschaften*", "M.Sc. Informationstechnologie"
)
# Farbkodierung für Studiengänge:  


studiengang_verteilung["Abschlussart"] = studiengang_verteilung["Studiengang"].apply(
    lambda x: "Bachelor" if "B.Sc." in x else "Master"
)

# Farbzuordnung für die Abschlussarten
color_map = {"Bachelor": px.colors.qualitative.Safe[0], "Master":px.colors.qualitative.Safe[1]}

# Sortieren nach Anzahl der Studierenden
studiengang_verteilung = studiengang_verteilung.sort_values(by="Anzahl Studierende", ascending=False)

# Studiengänge als kategorische Achse setzen
studiengang_verteilung["Studiengang"] = pd.Categorical(
    studiengang_verteilung["Studiengang"],
    categories=studiengang_verteilung["Studiengang"],  # Reihenfolge aus sortiertem DataFrame übernehmen
    ordered=True
)

In [443]:
color_map

{'Bachelor': 'rgb(136, 204, 238)', 'Master': 'rgb(204, 102, 119)'}

In [447]:
# Treemap erstellen
fig = px.bar(
    studiengang_verteilung,
    x="Studiengang",
    y="Anzahl Studierende",
    title="Long-Tail in der Studierendenverteilung: Wenige große, viele kleine Studiengänge",
    subtitle="Ergebnisse der Studierendenbefragung 2024 im FB09",
    color_discrete_map=color_map,
    color="Abschlussart",
    template="infoviz"
)
fig.update_layout(
    margin=dict(b=190, r=130),
    xaxis=dict(title=""),
    height=600,
    showlegend=True
)

fig.show()

In [455]:
import plotly.express as px
import pandas as pd

# Neue Spalte für Bachelor/Master erstellen
studiengang_verteilung["Abschlussart"] = studiengang_verteilung["Studiengang"].apply(
    lambda x: "Bachelor" if "B.Sc." in x else "Master"
)

# Farbzuordnung
color_map = {"Bachelor": "rgb(136, 204, 238)", "Master": "rgb(204, 102, 119)"}

# Sortieren nach Anzahl der Studierenden
studiengang_verteilung = studiengang_verteilung.sort_values(by="Anzahl Studierende", ascending=False)

# Studiengänge als kategorische Achse setzen
studiengang_verteilung["Studiengang"] = pd.Categorical(
    studiengang_verteilung["Studiengang"],
    categories=studiengang_verteilung["Studiengang"],  # Reihenfolge aus sortiertem DataFrame übernehmen
    ordered=True
)

# Bar-Plot
fig = px.bar(
    studiengang_verteilung,
    x="Studiengang",
    y="Anzahl Studierende",
    color="Abschlussart",
    color_discrete_map=color_map,
    title="Long-Tail in der Studierendenverteilung: Wenige große, viele kleine Studiengänge",
    subtitle="Ergebnisse der Studierendenbefragung 2024 im FB09",
    template="infoviz"
)

# Achsen-Beschriftung optimieren
fig.update_layout(
    margin=dict(b=190, r=130),
    xaxis_title="",
    height=600,
    xaxis=dict(categoryorder="array", categoryarray=studiengang_verteilung["Studiengang"]),
    legend=dict(
        title="",
        orientation="h",
        yanchor="top",
        y=1.1,
        xanchor="right",
        x=1
    )
)

fig.show()

In [406]:
import pandas as pd
import plotly.express as px

# Sicherstellen, dass die Spalte "Anzahl Studierende" numerisch ist
studiengang_verteilung["Anzahl Studierende"] = pd.to_numeric(studiengang_verteilung["Anzahl Studierende"], errors="coerce").fillna(0)

# Die vier größten Studiengänge identifizieren
top4_studiengaenge = studiengang_verteilung.nlargest(4, "Anzahl Studierende")["Studiengang"].tolist()

# Farbzuordnung: Top 4 in festen Farben, Rest in Grau
color_map = {studiengang: px.colors.qualitative.Safe[i] for i, studiengang in enumerate(top4_studiengaenge)}
color_map.update({studiengang: "lightgrey" for studiengang in studiengang_verteilung["Studiengang"] if studiengang not in top4_studiengaenge})

# Pie-Chart erstellen
fig = px.pie(
    studiengang_verteilung,
    values="Anzahl Studierende",
    names="Studiengang",
    title="Die vier größten Studiengänge machen über 50% aller Studierenden aus",
    subtitle="Ergebnisse der Studierendenbefragung 2024 im FB09",
    color="Studiengang",
    color_discrete_map=color_map,
    template="infoviz",
    hole=0.2
)


# Schriftgrößen und Layout anpassen
fig.update_layout(
    font=dict(size=18),  # Generelle Schriftgröße
    width=1400,  # Breite des Diagramms
    height=800,   # Höhe des Diagramms
    showlegend=False,  # Legende ausblenden
    title=dict(
        font=dict(size=20, family="Arial", weight="bold", color="black"),
        xanchor="left",  
        xref="container",
        x=0.08,
        subtitle=dict(
            font=dict(color="gray", size=13),
        ),
    ),
)

# Labels innerhalb des Pie-Charts entfernen & Linienfarbe setzen
fig.update_traces(
    textinfo="none",  # Keine direkten Labels
    marker=dict(line=dict(color="black", width=1))  # Schwarze Trennlinien zwischen Segmenten
)

studiengang = "B.Sc. Ernährungswissenschaften"
value = studiengang_verteilung.loc[studiengang_verteilung["Studiengang"] == studiengang, "Anzahl Studierende"]    
fig.add_annotation(
    text=f"{studiengang}<br>{int(value)} Studierende",  
    x=0.72, 
    y=0.9,  
    showarrow=False,
    font=dict(size=14, color="black",),
    align="left",
    xanchor="left"
)

studiengang = "B.Sc. Ökotrophologie"
value = studiengang_verteilung.loc[studiengang_verteilung["Studiengang"] == studiengang, "Anzahl Studierende"]    
fig.add_annotation(
    text=f"{studiengang}<br> {int(value)} Studierende",  
    x=0.17, 
    y=0.9,  
    showarrow=False,
    font=dict(size=14, color="black",),
    align="right",
    xanchor="left"
)

studiengang = "M.Sc. Ernährungswissenschaften"
value = studiengang_verteilung.loc[studiengang_verteilung["Studiengang"] == studiengang, "Anzahl Studierende"]    
fig.add_annotation(
    text=f"{studiengang}<br> {int(value)} Studierende",  
    x=0.04, 
    y=0.5,  
    showarrow=False,
    font=dict(size=14, color="black",),
    align="right",
    xanchor="left"
)

studiengang = "B.Sc. Agrarwissenschaften"
value = studiengang_verteilung.loc[studiengang_verteilung["Studiengang"] == studiengang, "Anzahl Studierende"]    
fig.add_annotation(
    text=f"{studiengang}<br> {int(value)} Studierende",  
    x=0.12, 
    y=0.2,  
    showarrow=False,
    font=dict(size=14, color="black",),
    align="right",
    xanchor="left",
)


value = studiengang_verteilung.loc[studiengang_verteilung["Studiengang"] == studiengang, "Anzahl Studierende"]    
fig.add_annotation(
    text=f"Andere<br>1102 Studierende",  
    x=0.73, 
    y=0.2,  
    showarrow=False,
    font=dict(size=14, color="black",),
    align="left",
    xanchor="left",
)

fig.show()

/var/folders/9v/2nh1cygd6qv3_k0knr9c4y_80000gn/T/ipykernel_45859/884453214.py:54: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/9v/2nh1cygd6qv3_k0knr9c4y_80000gn/T/ipykernel_45859/884453214.py:66: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/9v/2nh1cygd6qv3_k0knr9c4y_80000gn/T/ipykernel_45859/884453214.py:78: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

/var/folders/9v/2nh1cygd6qv3_k0knr9c4y_80000gn/T/ipykernel_45859/884453214.py:90: FutureWarning:

Calling int on a single element Series is deprecated and will raise a TypeError in the future. Use int(ser.iloc[0]) instead

